**DEVELOPED BY**

**PARTHA PRATIM RAY**

**https://github.com/ParthaPRay/**

**parthapratimray1986@gmail.com**

In [73]:
!pip install spacy
!pip install transformers
!pip install torch
!pip install thinc
###
!pip install requests

In [63]:
P!python -m spacy download en_core_web_trf


/usr/local/lib/python3.10/dist-packages/cupy/_environment.py:540: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy-cuda11x, cupy-cuda12x

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  warnings.warn(f'''
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 4.3 MB/s eta 0:00:00
✔ Download and installation successful
You

In [ ]:
### BASIC CODE
####### Simulated Weather Report Summarization

# Text summarizer

### facebook/bart-large-cnn

import spacy
from transformers import pipeline
from thinc.api import set_gpu_allocator, require_gpu
import logging

# Configure logging for debug messages
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

# Step 1: Use GPU if available
try:
    set_gpu_allocator("pytorch")
    require_gpu(0)
    logging.info("Using GPU for computation.")
except Exception as e:
    logging.warning("GPU not available or not configured: %s", e)

# Step 2: Load the spaCy language model
try:
    logging.info("Loading spaCy language model.")
    nlp = spacy.load("en_core_web_trf")  # Transformer-enabled spaCy model
    logging.info("spaCy model loaded successfully.")
except Exception as e:
    logging.error("Failed to load spaCy model: %s", e)
    raise

# Step 3: Load the BART summarization pipeline from Hugging Face
try:
    logging.info("Loading BART summarization pipeline.")
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    logging.info("BART pipeline loaded successfully.")
except Exception as e:
    logging.error("Failed to load BART model: %s", e)
    raise

# Step 4: Define the text summarization function
def summarize_text(article, max_length=130, min_length=30, debug=False):
    try:
        logging.info("Processing text with spaCy.")
        doc = nlp(article)
        logging.debug("spaCy processing complete. Extracting sentences.")

        sentences = [sent.text for sent in doc.sents]
        logging.debug("Extracted %d sentences from the article.", len(sentences))

        if debug:
            logging.debug("Sentences: %s", sentences)

        logging.info("Generating summary using BART model.")
        summary = summarizer(article, max_length=max_length, min_length=min_length, do_sample=False)

        logging.info("Summary generated successfully.")
        return summary[0]['summary_text']
    except Exception as e:
        logging.error("Error during summarization: %s", e)
        raise

# Step 5: Sample text input
ARTICLE = """Weather today is mostly cloudy with occasional rain showers. Temperatures will remain cool, ranging between 15°C to 20°C. Wind speeds are expected to reach up to 30 km/h in some regions. Tomorrow, conditions are likely to improve with more sunshine."""

if __name__ == "__main__":
    try:
        logging.info("Starting summarization task.")
        summary = summarize_text(ARTICLE, max_length=100, min_length=20, debug=True)
        print("\nSummary:")
        print(summary)
        logging.info("Summarization task completed successfully.")
    except Exception as e:
        logging.critical("Summarization task failed: %s", e)


/usr/local/lib/python3.10/dist-packages/thinc/shims/pytorch.py:253: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(filelike, map_location=dev


Summary:
Weather today is mostly cloudy with occasional rain showers. Temperatures will remain cool, ranging between 15°C to 20°C. Tomorrow, conditions are likely to improve with more sunshine.


In [66]:
!pip install requests

In [ ]:
!pip install cupy-cuda11x  # Replace `11x` with your CUDA version (e.g., `cuda11.8`, `cuda10.2`)


In [ ]:
################## NOAA v1 working

import spacy
from transformers import pipeline
from thinc.api import set_gpu_allocator, require_gpu
import logging
import requests
import sys
from datetime import datetime
import os

# Configure logging for debug messages
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Constants
NOAA_ENDPOINT = "https://api.weather.gov/points/{latitude},{longitude}"
USER_AGENT = "WeatherApp/1.0 (your.email@example.com)"  # Replace with your actual email

# Optional: Set Hugging Face token if available
# To set the token, add it to Colab secrets or set it directly in the notebook.
# For example, you can set it directly (not recommended for shared notebooks):
# os.environ["HF_TOKEN"] = "your_huggingface_token_here"

HF_TOKEN = os.getenv("HF_TOKEN")
if HF_TOKEN:
    os.environ["HUGGINGFACE_HUB_TOKEN"] = HF_TOKEN
    logging.info("Hugging Face token found and set.")
else:
    logging.warning("""
    Hugging Face token (HF_TOKEN) not found.
    Proceeding without authentication may lead to rate limiting.
    To set the token, add it to Colab secrets or set it directly in the notebook.
    """)

# Step 1: Use GPU if available
# Attempt to use GPU; if unavailable, fallback to CPU
try:
    set_gpu_allocator("pytorch")
    require_gpu(0)
    logging.info("Using GPU for computation.")
    device = 0  # GPU device
except Exception as e:
    logging.warning("GPU not available or not configured: %s", e)
    device = -1  # CPU device

# Step 2: Load the spaCy language model
try:
    logging.info("Loading spaCy language model.")
    nlp = spacy.load("en_core_web_trf")  # Transformer-enabled spaCy model
    logging.info("spaCy model loaded successfully.")
except Exception as e:
    logging.error("Failed to load spaCy model: %s", e)
    raise

# Step 3: Load the BART summarization pipeline from Hugging Face
try:
    logging.info("Loading BART summarization pipeline.")
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=device)
    logging.info("BART pipeline loaded successfully.")
except Exception as e:
    logging.error("Failed to load BART model: %s", e)
    raise

# Step 4: Define the function to fetch weather data from NOAA's NWS API
def fetch_weather_data(latitude, longitude):
    """
    Fetch weather forecast data from NOAA's NWS API for the given latitude and longitude.

    Args:
        latitude (float): Latitude of the location.
        longitude (float): Longitude of the location.

    Returns:
        dict: A dictionary containing the forecast data.
    """
    endpoint = NOAA_ENDPOINT.format(latitude=latitude, longitude=longitude)
    headers = {
        'User-Agent': USER_AGENT
    }

    try:
        logging.info("Fetching metadata from NOAA NWS API.")
        response = requests.get(endpoint, headers=headers, timeout=10)
        response.raise_for_status()
        data = response.json()

        # Extract the forecast URL
        forecast_url = data['properties']['forecast']

        logging.info("Fetching forecast data from NOAA NWS API.")
        forecast_response = requests.get(forecast_url, headers=headers, timeout=10)
        forecast_response.raise_for_status()
        forecast_data = forecast_response.json()

        return forecast_data

    except requests.exceptions.HTTPError as http_err:
        logging.error("HTTP error occurred: %s", http_err)
    except requests.exceptions.ConnectionError as conn_err:
        logging.error("Connection error occurred: %s", conn_err)
    except requests.exceptions.Timeout as timeout_err:
        logging.error("Timeout error occurred: %s", timeout_err)
    except requests.exceptions.RequestException as req_err:
        logging.error("Request exception: %s", req_err)
    except KeyError as key_err:
        logging.error("Key error: %s", key_err)
    except Exception as e:
        logging.error("An unexpected error occurred: %s", e)

    return None

# Step 5: Define the function to parse weather JSON into a coherent paragraph
def parse_weather_data(forecast_data):
    """
    Parse the NOAA NWS JSON response and construct a descriptive paragraph.

    Args:
        forecast_data (dict): JSON data from NOAA NWS API.

    Returns:
        str: A well-formed paragraph describing the weather forecast.
    """
    try:
        logging.info("Parsing weather data.")
        periods = forecast_data['properties']['periods']

        paragraphs = []

        for period in periods:
            name = period.get('name', 'N/A')
            temperature = period.get('temperature', 'N/A')
            temperature_unit = period.get('temperatureUnit', '')
            wind_speed = period.get('windSpeed', 'N/A')
            wind_direction = period.get('windDirection', 'N/A')
            short_forecast = period.get('shortForecast', '')
            detailed_forecast = period.get('detailedForecast', '')

            forecast_paragraph = (
                f"{name}: The temperature will be {temperature}°{temperature_unit} "
                f"with {short_forecast.lower()}. Winds are expected to be {wind_speed} "
                f"from the {wind_direction.lower()}. {detailed_forecast}"
            )
            paragraphs.append(forecast_paragraph)
            logging.debug("Parsed forecast for %s: %s", name, forecast_paragraph)

        full_paragraph = " ".join(paragraphs)
        logging.info("Weather data parsed into paragraph successfully.")
        return full_paragraph

    except KeyError as key_err:
        logging.error("Key error while parsing weather data: %s", key_err)
    except Exception as e:
        logging.error("Error parsing weather data: %s", e)

    return ""

# Step 6: Define the text summarization function
def summarize_text(article, max_length=150, min_length=40, debug=False):
    """
    Summarize the given text using the BART model.

    Args:
        article (str): The text to summarize.
        max_length (int): Maximum length of the summary.
        min_length (int): Minimum length of the summary.
        debug (bool): If True, enables debug logging.

    Returns:
        str: The summarized text.
    """
    try:
        logging.info("Processing text with spaCy.")
        doc = nlp(article)
        logging.debug("spaCy processing complete. Extracting sentences.")

        sentences = [sent.text for sent in doc.sents]
        logging.debug("Extracted %d sentences from the article.", len(sentences))

        if debug:
            logging.debug("Sentences: %s", sentences)

        logging.info("Generating summary using BART model.")
        summary = summarizer(
            article,
            max_length=max_length,
            min_length=min_length,
            do_sample=False
        )

        logging.info("Summary generated successfully.")
        return summary[0]['summary_text']
    except Exception as e:
        logging.error("Error during summarization: %s", e)
        raise

# Step 7: Main function to integrate all steps
def main(latitude, longitude):
    """
    Main function to fetch, parse, and summarize weather data.

    Args:
        latitude (float): Latitude of the location.
        longitude (float): Longitude of the location.
    """
    try:
        logging.info("Starting weather data retrieval and summarization.")

        # Fetch weather data
        forecast_data = fetch_weather_data(latitude, longitude)

        if not forecast_data:
            logging.warning("No forecast data retrieved.")
            return

        # Parse weather data into paragraph
        weather_paragraph = parse_weather_data(forecast_data)

        if not weather_paragraph:
            logging.warning("No weather information available to summarize.")
            return

        # Summarize the paragraph
        summary = summarize_text(weather_paragraph, max_length=150, min_length=40, debug=True)

        print("\nSummary:")
        print(summary)
        logging.info("Summarization task completed successfully.")

    except Exception as e:
        logging.critical("Summarization task failed: %s", e)

# Step 8: Run the main function with specified coordinates
# Define your latitude and longitude here
# Example: Dallas, TX (32.7767, -96.7970)
LATITUDE = 32.7767
LONGITUDE = -96.7970

# Alternatively, you can use input prompts to enter coordinates
# Uncomment the following lines to enable user input

# try:
#     LATITUDE = float(input("Enter latitude: "))
#     LONGITUDE = float(input("Enter longitude: "))
# except ValueError:
#     logging.error("Invalid input. Please enter numeric values for latitude and longitude.")
#     sys.exit(1)

# Execute the main function
main(LATITUDE, LONGITUDE)


    Hugging Face token (HF_TOKEN) not found.
    Proceeding without authentication may lead to rate limiting.
    To set the token, add it to Colab secrets or set it directly in the notebook.
    
Device set to use cpu



Summary:
Thursday: The temperature will be 62°F with patchy fog. Low around 53, with temperatures rising to around 55 overnight. A chance of rain showers and patchy Fog before 9am, then patchy. fog and showers and thunderstorms between 9am and noon. Friday: The weather will be mostly sunny, with a high near 68.


In [ ]:
!pip install gradio

In [ ]:
################# NOAAA v2 GRADIO

# Text summarization

# facebook/bart-large-cnn

# NOAA Raw Response + Sumarize weather forecast

import spacy
from transformers import pipeline
from thinc.api import set_gpu_allocator, require_gpu
import logging
import requests
import sys
from datetime import datetime
import os
import json
import gradio as gr

# Configure logging for debug messages
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Constants
NOAA_ENDPOINT = "https://api.weather.gov/points/{latitude},{longitude}"
USER_AGENT = "WeatherApp/1.0 (your.email@example.com)"  # Replace with your actual email

# Optional: Set Hugging Face token if available
# To set the token, add it to Colab secrets or set it directly in the notebook.
# For example, you can set it directly (not recommended for shared notebooks):
# os.environ["HF_TOKEN"] = "your_huggingface_token_here"

HF_TOKEN = os.getenv("HF_TOKEN")
if HF_TOKEN:
    os.environ["HUGGINGFACE_HUB_TOKEN"] = HF_TOKEN
    logging.info("Hugging Face token found and set.")
else:
    logging.warning("""
    Hugging Face token (HF_TOKEN) not found.
    Proceeding without authentication may lead to rate limiting.
    To set the token, add it to Colab secrets or set it directly in the notebook.
    """)

# Step 1: Use GPU if available
# Attempt to use GPU; if unavailable, fallback to CPU
try:
    set_gpu_allocator("pytorch")
    require_gpu(0)
    logging.info("Using GPU for computation.")
    device = 0  # GPU device
except Exception as e:
    logging.warning("GPU not available or not configured: %s", e)
    device = -1  # CPU device

# Step 2: Load the spaCy language model
try:
    logging.info("Loading spaCy language model.")
    nlp = spacy.load("en_core_web_trf")  # Transformer-enabled spaCy model
    logging.info("spaCy model loaded successfully.")
except Exception as e:
    logging.error("Failed to load spaCy model: %s", e)
    raise

# Step 3: Load the BART summarization pipeline from Hugging Face
try:
    logging.info("Loading BART summarization pipeline.")
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=device)
    logging.info("BART pipeline loaded successfully.")
except Exception as e:
    logging.error("Failed to load BART model: %s", e)
    raise

# Step 4: Define the function to fetch weather data from NOAA's NWS API
def fetch_weather_data(latitude, longitude):
    """
    Fetch weather forecast data from NOAA's NWS API for the given latitude and longitude.

    Args:
        latitude (float): Latitude of the location.
        longitude (float): Longitude of the location.

    Returns:
        dict: A dictionary containing the forecast data.
    """
    endpoint = NOAA_ENDPOINT.format(latitude=latitude, longitude=longitude)
    headers = {
        'User-Agent': USER_AGENT
    }

    try:
        logging.info("Fetching metadata from NOAA NWS API.")
        response = requests.get(endpoint, headers=headers, timeout=10)
        response.raise_for_status()
        data = response.json()
        logging.debug(f"Metadata Response: {json.dumps(data, indent=2)}")  # Debug message

        # Extract the forecast URL
        forecast_url = data['properties']['forecast']

        logging.info("Fetching forecast data from NOAA NWS API.")
        forecast_response = requests.get(forecast_url, headers=headers, timeout=10)
        forecast_response.raise_for_status()
        forecast_data = forecast_response.json()
        logging.debug(f"Forecast Response: {json.dumps(forecast_data, indent=2)}")  # Debug message

        return forecast_data

    except requests.exceptions.HTTPError as http_err:
        logging.error(f"HTTP error occurred: {http_err}")
    except requests.exceptions.ConnectionError as conn_err:
        logging.error(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        logging.error(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        logging.error(f"Request exception: {req_err}")
    except KeyError as key_err:
        logging.error(f"Key error: {key_err}")
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")

    return None

# Step 5: Define the function to parse weather JSON into a coherent paragraph
def parse_weather_data(forecast_data):
    """
    Parse the NOAA NWS JSON response and construct a descriptive paragraph.

    Args:
        forecast_data (dict): JSON data from NOAA NWS API.

    Returns:
        str: A well-formed paragraph describing the weather forecast.
    """
    try:
        logging.info("Parsing weather data.")
        periods = forecast_data['properties']['periods']

        paragraphs = []

        for period in periods:
            name = period.get('name', 'N/A')
            temperature = period.get('temperature', 'N/A')
            temperature_unit = period.get('temperatureUnit', '')
            wind_speed = period.get('windSpeed', 'N/A')
            wind_direction = period.get('windDirection', 'N/A')
            short_forecast = period.get('shortForecast', '')
            detailed_forecast = period.get('detailedForecast', '')

            forecast_paragraph = (
                f"{name}: The temperature will be {temperature}°{temperature_unit} "
                f"with {short_forecast.lower()}. Winds are expected to be {wind_speed} "
                f"from the {wind_direction.lower()}. {detailed_forecast}"
            )
            paragraphs.append(forecast_paragraph)
            logging.debug(f"Parsed forecast for {name}: {forecast_paragraph}")

        full_paragraph = " ".join(paragraphs)
        logging.info("Weather data parsed into paragraph successfully.")
        logging.debug(f"Full Paragraph: {full_paragraph}")  # Debug message
        return full_paragraph

    except KeyError as key_err:
        logging.error(f"Key error while parsing weather data: {key_err}")
    except Exception as e:
        logging.error(f"Error parsing weather data: {e}")

    return ""

# Step 6: Define the text summarization function
def summarize_text(article, max_length=150, min_length=40, debug=False):
    """
    Summarize the given text using the BART model.

    Args:
        article (str): The text to summarize.
        max_length (int): Maximum length of the summary.
        min_length (int): Minimum length of the summary.
        debug (bool): If True, enables debug logging.

    Returns:
        str: The summarized text.
    """
    try:
        logging.info("Processing text with spaCy.")
        doc = nlp(article)
        logging.debug("spaCy processing complete. Extracting sentences.")

        sentences = [sent.text for sent in doc.sents]
        logging.debug(f"Extracted {len(sentences)} sentences from the article.")

        if debug:
            logging.debug(f"Sentences: {sentences}")

        logging.info("Generating summary using BART model.")
        summary = summarizer(
            article,
            max_length=max_length,
            min_length=min_length,
            do_sample=False
        )

        logging.info("Summary generated successfully.")
        return summary[0]['summary_text']
    except Exception as e:
        logging.error(f"Error during summarization: {e}")
        raise

# Step 7: Define the Gradio interface function
def weather_summarizer(latitude, longitude):
    """
    Gradio interface function to fetch, parse, and summarize weather data.

    Args:
        latitude (float): Latitude of the location.
        longitude (float): Longitude of the location.

    Returns:
        tuple: Raw weather data (JSON string) and summarized weather forecast.
    """
    # Fetch weather data
    forecast_data = fetch_weather_data(latitude, longitude)

    if not forecast_data:
        return "No forecast data retrieved.", "Unable to generate summary."

    # Parse weather data into paragraph
    weather_paragraph = parse_weather_data(forecast_data)

    if not weather_paragraph:
        return "No weather information available to summarize.", "Unable to generate summary."

    # Summarize the paragraph
    summary = summarize_text(weather_paragraph, max_length=150, min_length=40, debug=True)

    # Prepare raw data for display (pretty-printed JSON)
    raw_data = json.dumps(forecast_data, indent=2)

    return raw_data, summary

# Step 8: Create the Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# Weather Summarizer")
    gr.Markdown("Enter the latitude and longitude of a location to get a summarized weather forecast.")

    with gr.Row():
        latitude_input = gr.Number(label="Latitude", value=32.7767)
        longitude_input = gr.Number(label="Longitude", value=-96.7970)

    submit_button = gr.Button("Submit")

    with gr.Row():
        raw_output = gr.Textbox(label="Raw NOAA API Response", lines=20)
        summary_output = gr.Textbox(label="Summarized Weather Forecast", lines=10)

    submit_button.click(
        fn=weather_summarizer,
        inputs=[latitude_input, longitude_input],
        outputs=[raw_output, summary_output]
    )

    gr.Markdown("""
    ---
    **Note:** Ensure that the latitude and longitude values are valid. For example:
    - **Dallas, TX:** Latitude `32.7767`, Longitude `-96.7970`
    - **New York City, NY:** Latitude `40.7128`, Longitude `-74.0060`
    """)

# Step 9: Launch the Gradio interface
demo.launch()


    Hugging Face token (HF_TOKEN) not found.
    Proceeding without authentication may lead to rate limiting.
    To set the token, add it to Colab secrets or set it directly in the notebook.
    
Device set to use cpu


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e2ef4d098b68eaa276.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
############### NOAA V2

# Text summarization

# facebook/bart-large-cnn

# Raw data + Raw summary + Temperature plot

import os
import json
import logging
from typing import Optional, Tuple, List
from datetime import datetime
import matplotlib.pyplot as plt

import requests
from cachetools import TTLCache, cached
import spacy
from transformers import pipeline
import gradio as gr

# ---------------------- Configuration ----------------------

# Constants and configuration variables
USER_AGENT = "WeatherApp/1.0 (your.email@example.com)"  # Replace with your actual email
CACHE_TTL = 600  # Cache Time-To-Live in seconds (e.g., 10 minutes)
FORECAST_DAYS = 3  # Number of days to include in the summary

# Optional: Hugging Face Token for authenticated access
HF_TOKEN = os.getenv("HF_TOKEN")  # Ensure this environment variable is set if you have a token

# ---------------------- Logging Setup ----------------------

def setup_logging():
    """
    Configures the logging settings.
    """
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s'
    )

setup_logging()

# ---------------------- Cache Setup ----------------------

# Initialize a TTL cache with a maximum size of 100 items and TTL of CACHE_TTL seconds
forecast_cache = TTLCache(maxsize=100, ttl=CACHE_TTL)

# ---------------------- NOAA Data Fetcher ----------------------

class NOAADataFetcher:
    """
    Fetches weather forecast data from NOAA's National Weather Service (NWS) API.
    """

    def __init__(self, user_agent: str):
        self.user_agent = user_agent
        self.headers = {'User-Agent': self.user_agent}
        self.endpoint_template = "https://api.weather.gov/points/{latitude},{longitude}"

    @cached(cache=forecast_cache)
    def fetch_forecast(self, latitude: float, longitude: float) -> Optional[dict]:
        """
        Fetches the weather forecast data for the given latitude and longitude.

        Args:
            latitude (float): Latitude of the location.
            longitude (float): Longitude of the location.

        Returns:
            dict: JSON response containing forecast data, or None if failed.
        """
        endpoint = self.endpoint_template.format(latitude=latitude, longitude=longitude)
        try:
            logging.info(f"Fetching metadata from NOAA NWS API for lat: {latitude}, lon: {longitude}.")
            response = requests.get(endpoint, headers=self.headers, timeout=10)
            response.raise_for_status()
            data = response.json()
            logging.debug(f"Metadata Response: {json.dumps(data, indent=2)}")

            # Extract the forecast URL
            forecast_url = data['properties']['forecast']
            logging.info("Fetching forecast data from NOAA NWS API.")
            forecast_response = requests.get(forecast_url, headers=self.headers, timeout=10)
            forecast_response.raise_for_status()
            forecast_data = forecast_response.json()
            logging.debug(f"Forecast Response: {json.dumps(forecast_data, indent=2)}")

            return forecast_data

        except requests.exceptions.HTTPError as http_err:
            logging.error(f"HTTP error occurred: {http_err}")
        except requests.exceptions.RequestException as req_err:
            logging.error(f"Request exception: {req_err}")
        except KeyError as key_err:
            logging.error(f"Key error: {key_err}")
        except Exception as e:
            logging.error(f"An unexpected error occurred: {e}")

        return None

# ---------------------- Weather Data Parser ----------------------

class WeatherDataParser:
    """
    Parses and processes raw weather forecast data from NOAA's NWS API.
    """

    def parse_forecast(self, forecast_data: dict, days: int = 3) -> List[str]:
        """
        Parses forecast data and constructs descriptive paragraphs.

        Args:
            forecast_data (dict): JSON data from NOAA NWS API.
            days (int): Number of days to include in the summary.

        Returns:
            list: List of descriptive paragraphs for each forecast period.
        """
        try:
            logging.info("Parsing weather data.")
            periods = forecast_data['properties']['periods']
            filtered_periods = self.filter_forecast_periods(periods, days)
            paragraphs = self.construct_paragraphs(filtered_periods)
            logging.debug(f"Full Paragraph: {' '.join(paragraphs)}")
            return paragraphs
        except KeyError as key_err:
            logging.error(f"Key error while parsing weather data: {key_err}")
            raise
        except Exception as e:
            logging.error(f"Error parsing weather data: {e}")
            raise

    def filter_forecast_periods(self, periods: List[dict], days: int) -> List[dict]:
        """
        Filters forecast periods to include only the specified number of days.

        Args:
            periods (list): List of forecast periods.
            days (int): Number of days to include.

        Returns:
            list: Filtered list of forecast periods.
        """
        filtered = []
        current_day = None
        for period in periods:
            day = period['startTime'][:10]  # Extract YYYY-MM-DD
            if day != current_day:
                current_day = day
                filtered.append(period)
                if len(filtered) >= days * 2:  # Day and Night for each day
                    break
        logging.info(f"Filtered down to {len(filtered)} periods for summarization.")
        return filtered

    def construct_paragraphs(self, periods: List[dict]) -> List[str]:
        """
        Constructs descriptive paragraphs from forecast periods.

        Args:
            periods (list): List of forecast periods.

        Returns:
            list: List of descriptive paragraphs.
        """
        paragraphs = []
        for period in periods:
            name = period.get('name', 'N/A')
            temperature = period.get('temperature', 'N/A')
            temperature_unit = period.get('temperatureUnit', '')
            wind_speed = period.get('windSpeed', 'N/A')
            wind_direction = period.get('windDirection', 'N/A')
            short_forecast = period.get('shortForecast', '')
            detailed_forecast = period.get('detailedForecast', '')

            forecast_paragraph = (
                f"{name}: The temperature will be {temperature}°{temperature_unit} "
                f"with {short_forecast.lower()}. Winds are expected to be {wind_speed} "
                f"from the {wind_direction.lower()}. {detailed_forecast}"
            )
            paragraphs.append(forecast_paragraph)
            logging.debug(f"Parsed forecast for {name}: {forecast_paragraph}")
        return paragraphs

# ---------------------- Weather Summarizer ----------------------

class WeatherSummarizer:
    """
    Generates concise summaries from parsed weather forecast data using a summarization model.
    """

    def __init__(self, model_name: str = "facebook/bart-large-cnn", device: int = -1):
        """
        Initializes the summarizer with the specified model and device.

        Args:
            model_name (str): Name of the Hugging Face model to use.
            device (int): Device to run the model on (-1 for CPU, 0 for GPU).
        """
        try:
            logging.info("Loading spaCy language model for summarization.")
            self.nlp = spacy.load("en_core_web_trf")
            logging.info("spaCy model loaded successfully.")
        except Exception as e:
            logging.error(f"Failed to load spaCy model: {e}")
            raise

        try:
            logging.info(f"Loading BART summarization pipeline with model '{model_name}'.")
            self.summarizer = pipeline("summarization", model=model_name, device=device)
            logging.info("BART summarization pipeline loaded successfully.")
        except Exception as e:
            logging.error(f"Failed to load BART model: {e}")
            raise

    def summarize(self, paragraphs: List[str], max_length: int = 150, min_length: int = 40) -> str:
        """
        Generates a summary from the provided paragraphs.

        Args:
            paragraphs (list): List of descriptive paragraphs.
            max_length (int): Maximum length of the summary.
            min_length (int): Minimum length of the summary.

        Returns:
            str: Generated summary.
        """
        try:
            logging.info("Generating summary using BART model.")
            full_text = " ".join(paragraphs)
            summary = self.summarizer(
                full_text,
                max_length=max_length,
                min_length=min_length,
                do_sample=False
            )
            logging.info("Summary generated successfully.")
            return summary[0]['summary_text']
        except Exception as e:
            logging.error(f"Error during summarization: {e}")
            raise

# ---------------------- Visualization ----------------------

def plot_temperatures(paragraphs: List[str]) -> Optional[plt.Figure]:
    """
    Generates a temperature trend plot from the forecast paragraphs.

    Args:
        paragraphs (list): List of descriptive paragraphs.

    Returns:
        matplotlib.figure.Figure: Generated plot figure, or None if no data.
    """
    temperatures = []
    periods = []
    for p in paragraphs:
        try:
            temp_start = p.find("temperature will be ") + len("temperature will be ")
            temp_end = p.find("°", temp_start)
            temp = int(p[temp_start:temp_end])
            name = p.split(":")[0]
            temperatures.append(temp)
            periods.append(name)
        except:
            continue

    if not temperatures:
        logging.warning("No temperature data available for plotting.")
        return None

    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(periods, temperatures, marker='o', linestyle='-', color='b')
    ax.set_title('Temperature Over Time')
    ax.set_xlabel('Period')
    ax.set_ylabel('Temperature (°F)')
    ax.grid(True)
    plt.tight_layout()
    return fig

# ---------------------- Input Validation ----------------------

def validate_coordinates(latitude: float, longitude: float) -> Optional[str]:
    """
    Validates the latitude and longitude values.

    Args:
        latitude (float): Latitude value.
        longitude (float): Longitude value.

    Returns:
        str: Error message if invalid, else None.
    """
    errors = []
    if not (-90 <= latitude <= 90):
        errors.append("Latitude must be between -90 and 90.")
    if not (-180 <= longitude <= 180):
        errors.append("Longitude must be between -180 and 180.")
    if errors:
        return " ".join(errors)
    return None

# ---------------------- Gradio Interface Function ----------------------

def weather_gradio_interface(latitude: float, longitude: float, show_plot: bool) -> Tuple[str, str, Optional[plt.Figure]]:
    """
    Gradio interface function to fetch, parse, and summarize weather data.

    Args:
        latitude (float): Latitude input by the user.
        longitude (float): Longitude input by the user.
        show_plot (bool): Whether to display the temperature trend plot.

    Returns:
        tuple: Raw data (formatted JSON), summary, and plot (if enabled).
    """
    # Validate inputs
    validation_error = validate_coordinates(latitude, longitude)
    if validation_error:
        logging.error(f"Input validation failed: {validation_error}")
        return validation_error, "Unable to generate summary due to invalid inputs.", None

    # Initialize components
    fetcher = NOAADataFetcher(user_agent=USER_AGENT)
    parser = WeatherDataParser()
    summarizer = WeatherSummarizer(model_name="facebook/bart-large-cnn", device=0 if torch.cuda.is_available() else -1)

    try:
        # Fetch data
        forecast_data = fetcher.fetch_forecast(latitude, longitude)
        if not forecast_data:
            logging.error("No forecast data retrieved.")
            return "No forecast data retrieved.", "Unable to generate summary.", None

        # Parse data
        paragraphs = parser.parse_forecast(forecast_data, days=FORECAST_DAYS)
        if not paragraphs:
            logging.error("No weather information available to summarize.")
            return "No weather information available to summarize.", "Unable to generate summary.", None

        # Summarize
        summary = summarizer.summarize(paragraphs)

        # Prepare raw data
        raw_data = json.dumps(forecast_data, indent=2)

        # Plot temperatures if requested
        plot = plot_temperatures(paragraphs) if show_plot else None

        return raw_data, summary, plot

    except Exception as e:
        logging.critical(f"Error in Gradio interface: {e}")
        return f"An error occurred: {e}", "Unable to generate summary.", None

# ---------------------- Main Execution ----------------------

if __name__ == "__main__":
    # Ensure Hugging Face Token is set if available
    if HF_TOKEN:
        os.environ["HUGGINGFACE_HUB_TOKEN"] = HF_TOKEN
        logging.info("Hugging Face token found and set.")
    else:
        logging.warning("""
        Hugging Face token (HF_TOKEN) not found.
        Proceeding without authentication may lead to rate limiting.
        To set the token, add it to environment variables or set it directly in the notebook.
        """)

    # Determine device for transformers pipeline
    import torch
    device = 0 if torch.cuda.is_available() else -1
    if device == 0:
        logging.info("Using GPU for transformers pipeline.")
    else:
        logging.info("GPU not available. Using CPU for transformers pipeline.")

    # Launch Gradio Interface
    with gr.Blocks() as demo:
        gr.Markdown("# Weather Summarizer")
        gr.Markdown("""
        Enter the **latitude** and **longitude** of a location to retrieve a summarized weather forecast.
        """)

        with gr.Row():
            latitude_input = gr.Number(label="Latitude", value=32.7767)
            longitude_input = gr.Number(label="Longitude", value=-96.7970)

        show_plot = gr.Checkbox(label="Show Temperature Trend Plot", value=True)

        submit_button = gr.Button("Submit")

        with gr.Tab("Raw Data"):
            raw_output = gr.Textbox(label="Raw NOAA API Response", lines=20, interactive=False)

        with gr.Tab("Raw Summary"):
            summary_output = gr.Textbox(label="Summarized Weather Forecast", lines=10, interactive=False)

        with gr.Tab("Temperature Plot"):
            temperature_plot = gr.Plot(label="Temperature Trend")

        submit_button.click(
            fn=weather_gradio_interface,
            inputs=[latitude_input, longitude_input, show_plot],
            outputs=[raw_output, summary_output, temperature_plot]
        )

        gr.Markdown("""
        ---
        **Note:** Ensure that the latitude and longitude values are valid. For example:
        - **Dallas, TX:** Latitude `32.7767`, Longitude `-96.7970`
        - **New York City, NY:** Latitude `40.7128`, Longitude `-74.0060`
        """)

    demo.launch()


        Hugging Face token (HF_TOKEN) not found.
        Proceeding without authentication may lead to rate limiting.
        To set the token, add it to environment variables or set it directly in the notebook.
        


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f9e14041d9aa620745.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
############## NOAA V3

# Text summarization

# facebook/bart-large-cnn

# NOAA Raw Response + Sumarize weather forecast

# Detailed forecast + detailed summary



############### NOAA V2

# weather_summarizer.py

import os
import json
import logging
from typing import Optional, Tuple, List, Dict
from datetime import datetime
import matplotlib.pyplot as plt

import requests
from cachetools import TTLCache, cached
import spacy
from transformers import pipeline
import gradio as gr

# ---------------------- Configuration ----------------------

# Constants and configuration variables
USER_AGENT = "WeatherApp/1.0 (your.email@example.com)"  # Replace with your actual email
CACHE_TTL = 600  # Cache Time-To-Live in seconds (e.g., 10 minutes)
FORECAST_DAYS = 3  # Number of days to include in the summary

# Optional: Hugging Face Token for authenticated access
HF_TOKEN = os.getenv("HF_TOKEN")  # Ensure this environment variable is set if you have a token

# ---------------------- Logging Setup ----------------------

def setup_logging():
    """
    Configures the logging settings.
    """
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s'
    )

setup_logging()

# ---------------------- Cache Setup ----------------------

# Initialize a TTL cache with a maximum size of 100 items and TTL of CACHE_TTL seconds
forecast_cache = TTLCache(maxsize=100, ttl=CACHE_TTL)

# ---------------------- NOAA Data Fetcher ----------------------

class NOAADataFetcher:
    """
    Fetches weather forecast data from NOAA's National Weather Service (NWS) API.
    """

    def __init__(self, user_agent: str):
        self.user_agent = user_agent
        self.headers = {'User-Agent': self.user_agent}
        self.endpoint_template = "https://api.weather.gov/points/{latitude},{longitude}"

    @cached(cache=forecast_cache)
    def fetch_forecast(self, latitude: float, longitude: float) -> Optional[dict]:
        """
        Fetches the weather forecast data for the given latitude and longitude.

        Args:
            latitude (float): Latitude of the location.
            longitude (float): Longitude of the location.

        Returns:
            dict: JSON response containing forecast data, or None if failed.
        """
        endpoint = self.endpoint_template.format(latitude=latitude, longitude=longitude)
        try:
            logging.info(f"Fetching metadata from NOAA NWS API for lat: {latitude}, lon: {longitude}.")
            response = requests.get(endpoint, headers=self.headers, timeout=10)
            response.raise_for_status()
            data = response.json()
            logging.debug(f"Metadata Response: {json.dumps(data, indent=2)}")

            # Extract the forecast URL
            forecast_url = data['properties']['forecast']
            logging.info("Fetching forecast data from NOAA NWS API.")
            forecast_response = requests.get(forecast_url, headers=self.headers, timeout=10)
            forecast_response.raise_for_status()
            forecast_data = forecast_response.json()
            logging.debug(f"Forecast Response: {json.dumps(forecast_data, indent=2)}")

            return forecast_data

        except requests.exceptions.HTTPError as http_err:
            logging.error(f"HTTP error occurred: {http_err}")
        except requests.exceptions.RequestException as req_err:
            logging.error(f"Request exception: {req_err}")
        except KeyError as key_err:
            logging.error(f"Key error: {key_err}")
        except Exception as e:
            logging.error(f"An unexpected error occurred: {e}")

        return None

# ---------------------- Weather Data Parser ----------------------

class WeatherDataParser:
    """
    Parses and processes raw weather forecast data from NOAA's NWS API.
    """

    def parse_forecast(self, forecast_data: dict, days: int = 3) -> List[str]:
        """
        Parses forecast data and constructs descriptive paragraphs.

        Args:
            forecast_data (dict): JSON data from NOAA NWS API.
            days (int): Number of days to include in the summary.

        Returns:
            list: List of descriptive paragraphs for each forecast period.
        """
        try:
            logging.info("Parsing weather data.")
            periods = forecast_data['properties']['periods']
            filtered_periods = self.filter_forecast_periods(periods, days)
            paragraphs = self.construct_paragraphs(filtered_periods)
            logging.debug(f"Full Paragraph: {' '.join(paragraphs)}")
            return paragraphs
        except KeyError as key_err:
            logging.error(f"Key error while parsing weather data: {key_err}")
            raise
        except Exception as e:
            logging.error(f"Error parsing weather data: {e}")
            raise

    def filter_forecast_periods(self, periods: List[dict], days: int) -> List[dict]:
        """
        Filters forecast periods to include only the specified number of days.

        Args:
            periods (list): List of forecast periods.
            days (int): Number of days to include.

        Returns:
            list: Filtered list of forecast periods.
        """
        filtered = []
        current_day = None
        for period in periods:
            day = period['startTime'][:10]  # Extract YYYY-MM-DD
            if day != current_day:
                current_day = day
                filtered.append(period)
                if len(filtered) >= days * 2:  # Day and Night for each day
                    break
        logging.info(f"Filtered down to {len(filtered)} periods for summarization.")
        return filtered

    def construct_paragraphs(self, periods: List[dict]) -> List[str]:
        """
        Constructs descriptive paragraphs from forecast periods.

        Args:
            periods (list): List of forecast periods.

        Returns:
            list: List of descriptive paragraphs.
        """
        paragraphs = []
        for period in periods:
            name = period.get('name', 'N/A')
            temperature = period.get('temperature', 'N/A')
            temperature_unit = period.get('temperatureUnit', '')
            wind_speed = period.get('windSpeed', 'N/A')
            wind_direction = period.get('windDirection', 'N/A')
            short_forecast = period.get('shortForecast', '')
            detailed_forecast = period.get('detailedForecast', '')

            forecast_paragraph = (
                f"{name}: The temperature will be {temperature}°{temperature_unit} "
                f"with {short_forecast.lower()}. Winds are expected to be {wind_speed} "
                f"from the {wind_direction.lower()}. {detailed_forecast}"
            )
            paragraphs.append(forecast_paragraph)
            logging.debug(f"Parsed forecast for {name}: {forecast_paragraph}")
        return paragraphs

    ### New: Extract and Merge detailedForecasts

    def extract_detailed_forecasts(self, forecast_data: dict) -> List[Dict[str, str]]:
        """
        Extracts the 'number' and 'detailedForecast' from each forecast period.

        Args:
            forecast_data (dict): JSON data from NOAA NWS API.

        Returns:
            list: List of dictionaries containing 'number' and 'detailedForecast'.
        """
        try:
            logging.info("Extracting detailed forecasts.")
            periods = forecast_data['properties']['periods']
            detailed_forecasts = []
            for period in periods:
                number = period.get('number')
                detailed_forecast = period.get('detailedForecast', '')
                if number is not None:
                    detailed_forecasts.append({
                        "number": number,
                        "detailedForecast": detailed_forecast
                    })
            logging.debug(f"Extracted Detailed Forecasts: {detailed_forecasts}")
            return detailed_forecasts
        except KeyError as key_err:
            logging.error(f"Key error while extracting detailed forecasts: {key_err}")
            raise
        except Exception as e:
            logging.error(f"Error extracting detailed forecasts: {e}")
            raise

    def merge_detailed_forecasts(self, detailed_forecasts: List[Dict[str, str]]) -> str:
        """
        Merges the list of detailed forecasts into a structured JSON string.

        Args:
            detailed_forecasts (list): List of dictionaries with 'number' and 'detailedForecast'.

        Returns:
            str: Merged JSON string.
        """
        try:
            logging.info("Merging detailed forecasts into structured JSON.")
            merged_content = ""
            for forecast in detailed_forecasts:
                merged_content += f'"number": {forecast["number"]}\n"detailedForecast": "{forecast["detailedForecast"]}"\n'
            logging.debug(f"Merged Content: {merged_content}")
            return merged_content
        except Exception as e:
            logging.error(f"Error merging detailed forecasts: {e}")
            raise

# ---------------------- Weather Summarizer ----------------------

class WeatherSummarizer:
    """
    Generates concise summaries from parsed weather forecast data using a summarization model.
    """

    def __init__(self, model_name: str = "facebook/bart-large-cnn", device: int = -1):
        """
        Initializes the summarizer with the specified model and device.

        Args:
            model_name (str): Name of the Hugging Face model to use.
            device (int): Device to run the model on (-1 for CPU, 0 for GPU).
        """
        try:
            logging.info("Loading spaCy language model for summarization.")
            self.nlp = spacy.load("en_core_web_trf")
            logging.info("spaCy model loaded successfully.")
        except Exception as e:
            logging.error(f"Failed to load spaCy model: {e}")
            raise

        try:
            logging.info(f"Loading BART summarization pipeline with model '{model_name}'.")
            self.summarizer = pipeline("summarization", model=model_name, device=device)
            logging.info("BART summarization pipeline loaded successfully.")
        except Exception as e:
            logging.error(f"Failed to load BART model: {e}")
            raise

    def summarize(self, paragraphs: List[str], max_length: int = 150, min_length: int = 40) -> str:
        """
        Generates a summary from the provided paragraphs.

        Args:
            paragraphs (list): List of descriptive paragraphs.
            max_length (int): Maximum length of the summary.
            min_length (int): Minimum length of the summary.

        Returns:
            str: Generated summary.
        """
        try:
            logging.info("Generating summary using BART model.")
            full_text = " ".join(paragraphs)
            summary = self.summarizer(
                full_text,
                max_length=max_length,
                min_length=min_length,
                do_sample=False
            )
            logging.info("Summary generated successfully.")
            return summary[0]['summary_text']
        except Exception as e:
            logging.error(f"Error during summarization: {e}")
            raise

    ### New: Summarize Merged Detailed Forecasts

    def summarize_merged_forecast(self, merged_content: str, max_length: int = 150, min_length: int = 40) -> str:
        """
        Generates a summary from the merged detailed forecasts.

        Args:
            merged_content (str): Merged detailed forecasts as a single string.
            max_length (int): Maximum length of the summary.
            min_length (int): Minimum length of the summary.

        Returns:
            str: Generated summary.
        """
        try:
            logging.info("Generating summary for merged detailed forecasts.")
            # Process the merged content with spaCy
            doc = self.nlp(merged_content)
            sentences = [sent.text for sent in doc.sents]
            paragraph = " ".join(sentences)
            logging.debug(f"Processed Paragraph for Summarization: {paragraph}")

            # Generate summary
            summary = self.summarizer(
                paragraph,
                max_length=max_length,
                min_length=min_length,
                do_sample=False
            )
            logging.info("Merged summary generated successfully.")
            return summary[0]['summary_text']
        except Exception as e:
            logging.error(f"Error during merged summarization: {e}")
            raise

# ---------------------- Visualization ----------------------

def plot_temperatures(paragraphs: List[str]) -> Optional[plt.Figure]:
    """
    Generates a temperature trend plot from the forecast paragraphs.

    Args:
        paragraphs (list): List of descriptive paragraphs.

    Returns:
        matplotlib.figure.Figure: Generated plot figure, or None if no data.
    """
    temperatures = []
    periods = []
    for p in paragraphs:
        try:
            temp_start = p.find("temperature will be ") + len("temperature will be ")
            temp_end = p.find("°", temp_start)
            temp = int(p[temp_start:temp_end])
            name = p.split(":")[0]
            temperatures.append(temp)
            periods.append(name)
        except:
            continue

    if not temperatures:
        logging.warning("No temperature data available for plotting.")
        return None

    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(periods, temperatures, marker='o', linestyle='-', color='b')
    ax.set_title('Temperature Over Time')
    ax.set_xlabel('Period')
    ax.set_ylabel('Temperature (°F)')
    ax.grid(True)
    plt.tight_layout()
    return fig

# ---------------------- Input Validation ----------------------

def validate_coordinates(latitude: float, longitude: float) -> Optional[str]:
    """
    Validates the latitude and longitude values.

    Args:
        latitude (float): Latitude value.
        longitude (float): Longitude value.

    Returns:
        str: Error message if invalid, else None.
    """
    errors = []
    if not (-90 <= latitude <= 90):
        errors.append("Latitude must be between -90 and 90.")
    if not (-180 <= longitude <= 180):
        errors.append("Longitude must be between -180 and 180.")
    if errors:
        return " ".join(errors)
    return None

# ---------------------- Gradio Interface Function ----------------------

def weather_gradio_interface(latitude: float, longitude: float, show_plot: bool) -> Tuple[str, str, Optional[plt.Figure], str, str]:
    """
    Gradio interface function to fetch, parse, and summarize weather data.

    Args:
        latitude (float): Latitude input by the user.
        longitude (float): Longitude input by the user.
        show_plot (bool): Whether to display the temperature trend plot.

    Returns:
        tuple: Raw data (formatted JSON), summary, plot (if enabled), merged detailed forecasts, merged summary.
    """
    # Validate inputs
    validation_error = validate_coordinates(latitude, longitude)
    if validation_error:
        logging.error(f"Input validation failed: {validation_error}")
        return validation_error, "Unable to generate summary due to invalid inputs.", None, "", ""

    # Initialize components
    fetcher = NOAADataFetcher(user_agent=USER_AGENT)
    parser = WeatherDataParser()
    summarizer = WeatherSummarizer(model_name="facebook/bart-large-cnn", device=0 if torch.cuda.is_available() else -1)

    try:
        # Fetch data
        forecast_data = fetcher.fetch_forecast(latitude, longitude)
        if not forecast_data:
            logging.error("No forecast data retrieved.")
            return "No forecast data retrieved.", "Unable to generate summary.", None, "", ""

        # Parse data
        paragraphs = parser.parse_forecast(forecast_data, days=FORECAST_DAYS)
        if not paragraphs:
            logging.error("No weather information available to summarize.")
            return "No weather information available to summarize.", "Unable to generate summary.", None, "", ""

        # Summarize
        summary = summarizer.summarize(paragraphs)

        # Prepare raw data
        raw_data = json.dumps(forecast_data, indent=2)

        # Plot temperatures if requested
        plot = plot_temperatures(paragraphs) if show_plot else None

        ### New: Extract and Merge Detailed Forecasts
        detailed_forecasts = parser.extract_detailed_forecasts(forecast_data)
        merged_detailed_forecasts = parser.merge_detailed_forecasts(detailed_forecasts)

        # Summarize merged detailed forecasts
        merged_summary = summarizer.summarize_merged_forecast(merged_detailed_forecasts)

        return raw_data, summary, plot, merged_detailed_forecasts, merged_summary

    except Exception as e:
        logging.critical(f"Error in Gradio interface: {e}")
        return f"An error occurred: {e}", "Unable to generate summary.", None, "", ""

# ---------------------- Main Execution ----------------------

if __name__ == "__main__":
    # Ensure Hugging Face Token is set if available
    if HF_TOKEN:
        os.environ["HUGGINGFACE_HUB_TOKEN"] = HF_TOKEN
        logging.info("Hugging Face token found and set.")
    else:
        logging.warning("""
        Hugging Face token (HF_TOKEN) not found.
        Proceeding without authentication may lead to rate limiting.
        To set the token, add it to environment variables or set it directly in the notebook.
        """)

    # Determine device for transformers pipeline
    import torch
    device = 0 if torch.cuda.is_available() else -1
    if device == 0:
        logging.info("Using GPU for transformers pipeline.")
    else:
        logging.info("GPU not available. Using CPU for transformers pipeline.")

    # Launch Gradio Interface
    with gr.Blocks() as demo:
        gr.Markdown("# Weather Summarizer")
        gr.Markdown("""
        Enter the **latitude** and **longitude** of a location to retrieve a summarized weather forecast.
        """)

        with gr.Row():
            latitude_input = gr.Number(label="Latitude", value=32.7767)
            longitude_input = gr.Number(label="Longitude", value=-96.7970)

        show_plot = gr.Checkbox(label="Show Temperature Trend Plot", value=True)

        submit_button = gr.Button("Submit")

        with gr.Tab("Raw Data"):
            raw_output = gr.Textbox(label="Raw NOAA API Response", lines=20, interactive=False)

        with gr.Tab("Raw Summary"):
            summary_output = gr.Textbox(label="Summarized Weather Forecast", lines=10, interactive=False)

        with gr.Tab("Temperature Plot"):
            temperature_plot = gr.Plot(label="Temperature Trend")

        ### New: Additional Tabs for Merged Forecasts and Summary
        with gr.Tab("Detailed Forecasts"):
            merged_forecast_output = gr.Textbox(label="Merged Detailed Forecasts", lines=30, interactive=False)

        with gr.Tab("Detailed Summary"):
            merged_summary_output = gr.Textbox(label="Summarized Merged Forecast", lines=5, interactive=False)

        submit_button.click(
            fn=weather_gradio_interface,
            inputs=[latitude_input, longitude_input, show_plot],
            outputs=[raw_output, summary_output, temperature_plot, merged_forecast_output, merged_summary_output]
        )

        gr.Markdown("""
        ---
        **Note:** Ensure that the latitude and longitude values are valid. For example:
        - **Dallas, TX:** Latitude `32.7767`, Longitude `-96.7970`
        - **New York City, NY:** Latitude `40.7128`, Longitude `-74.0060`
        """)

    demo.launch()


        Hugging Face token (HF_TOKEN) not found.
        Proceeding without authentication may lead to rate limiting.
        To set the token, add it to environment variables or set it directly in the notebook.
        


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://20327786ded22d3589.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
############## NOAA V4


# Text summarization

# facebook/bart-large-cnn

# NOAA Raw Response + Sumarize weather forecast

# Detailed forecast + detailed summary

# Short forecat + short summary

import os
import json
import logging
from typing import Optional, Tuple, List, Dict
from datetime import datetime
import matplotlib.pyplot as plt

import requests
from cachetools import TTLCache, cached
import spacy
from transformers import pipeline
import gradio as gr

# ---------------------- Configuration ----------------------

# Constants and configuration variables
USER_AGENT = "WeatherApp/1.0 (your.email@example.com)"  # Replace with your actual email
CACHE_TTL = 600  # Cache Time-To-Live in seconds (e.g., 10 minutes)
FORECAST_DAYS = 3  # Number of days to include in the summary

# Optional: Hugging Face Token for authenticated access
HF_TOKEN = os.getenv("HF_TOKEN")  # Ensure this environment variable is set if you have a token

# ---------------------- Logging Setup ----------------------

def setup_logging():
    """
    Configures the logging settings.
    """
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s'
    )

setup_logging()

# ---------------------- Cache Setup ----------------------

# Initialize a TTL cache with a maximum size of 100 items and TTL of CACHE_TTL seconds
forecast_cache = TTLCache(maxsize=100, ttl=CACHE_TTL)

# ---------------------- NOAA Data Fetcher ----------------------

class NOAADataFetcher:
    """
    Fetches weather forecast data from NOAA's National Weather Service (NWS) API.
    """

    def __init__(self, user_agent: str):
        self.user_agent = user_agent
        self.headers = {'User-Agent': self.user_agent}
        self.endpoint_template = "https://api.weather.gov/points/{latitude},{longitude}"

    @cached(cache=forecast_cache)
    def fetch_forecast(self, latitude: float, longitude: float) -> Optional[dict]:
        """
        Fetches the weather forecast data for the given latitude and longitude.

        Args:
            latitude (float): Latitude of the location.
            longitude (float): Longitude of the location.

        Returns:
            dict: JSON response containing forecast data, or None if failed.
        """
        endpoint = self.endpoint_template.format(latitude=latitude, longitude=longitude)
        try:
            logging.info(f"Fetching metadata from NOAA NWS API for lat: {latitude}, lon: {longitude}.")
            response = requests.get(endpoint, headers=self.headers, timeout=10)
            response.raise_for_status()
            data = response.json()
            logging.debug(f"Metadata Response: {json.dumps(data, indent=2)}")

            # Extract the forecast URL
            forecast_url = data['properties']['forecast']
            logging.info("Fetching forecast data from NOAA NWS API.")
            forecast_response = requests.get(forecast_url, headers=self.headers, timeout=10)
            forecast_response.raise_for_status()
            forecast_data = forecast_response.json()
            logging.debug(f"Forecast Response: {json.dumps(forecast_data, indent=2)}")

            return forecast_data

        except requests.exceptions.HTTPError as http_err:
            logging.error(f"HTTP error occurred: {http_err}")
        except requests.exceptions.RequestException as req_err:
            logging.error(f"Request exception: {req_err}")
        except KeyError as key_err:
            logging.error(f"Key error: {key_err}")
        except Exception as e:
            logging.error(f"An unexpected error occurred: {e}")

        return None

# ---------------------- Weather Data Parser ----------------------

class WeatherDataParser:
    """
    Parses and processes raw weather forecast data from NOAA's NWS API.
    """

    def parse_forecast(self, forecast_data: dict, days: int = 3) -> List[str]:
        """
        Parses forecast data and constructs descriptive paragraphs.

        Args:
            forecast_data (dict): JSON data from NOAA NWS API.
            days (int): Number of days to include in the summary.

        Returns:
            list: List of descriptive paragraphs for each forecast period.
        """
        try:
            logging.info("Parsing weather data.")
            periods = forecast_data['properties']['periods']
            filtered_periods = self.filter_forecast_periods(periods, days)
            paragraphs = self.construct_paragraphs(filtered_periods)
            logging.debug(f"Full Paragraph: {' '.join(paragraphs)}")
            return paragraphs
        except KeyError as key_err:
            logging.error(f"Key error while parsing weather data: {key_err}")
            raise
        except Exception as e:
            logging.error(f"Error parsing weather data: {e}")
            raise

    def filter_forecast_periods(self, periods: List[dict], days: int) -> List[dict]:
        """
        Filters forecast periods to include only the specified number of days.

        Args:
            periods (list): List of forecast periods.
            days (int): Number of days to include.

        Returns:
            list: Filtered list of forecast periods.
        """
        filtered = []
        current_day = None
        for period in periods:
            day = period['startTime'][:10]  # Extract YYYY-MM-DD
            if day != current_day:
                current_day = day
                filtered.append(period)
                if len(filtered) >= days * 2:  # Day and Night for each day
                    break
        logging.info(f"Filtered down to {len(filtered)} periods for summarization.")
        return filtered

    def construct_paragraphs(self, periods: List[dict]) -> List[str]:
        """
        Constructs descriptive paragraphs from forecast periods.

        Args:
            periods (list): List of forecast periods.

        Returns:
            list: List of descriptive paragraphs.
        """
        paragraphs = []
        for period in periods:
            name = period.get('name', 'N/A')
            temperature = period.get('temperature', 'N/A')
            temperature_unit = period.get('temperatureUnit', '')
            wind_speed = period.get('windSpeed', 'N/A')
            wind_direction = period.get('windDirection', 'N/A')
            short_forecast = period.get('shortForecast', '')
            detailed_forecast = period.get('detailedForecast', '')

            forecast_paragraph = (
                f"{name}: The temperature will be {temperature}°{temperature_unit} "
                f"with {short_forecast.lower()}. Winds are expected to be {wind_speed} "
                f"from the {wind_direction.lower()}. {detailed_forecast}"
            )
            paragraphs.append(forecast_paragraph)
            logging.debug(f"Parsed forecast for {name}: {forecast_paragraph}")
        return paragraphs

    ### New: Extract and Merge detailedForecasts

    def extract_detailed_forecasts(self, forecast_data: dict) -> List[Dict[str, str]]:
        """
        Extracts the 'number' and 'detailedForecast' from each forecast period.

        Args:
            forecast_data (dict): JSON data from NOAA NWS API.

        Returns:
            list: List of dictionaries containing 'number' and 'detailedForecast'.
        """
        try:
            logging.info("Extracting detailed forecasts.")
            periods = forecast_data['properties']['periods']
            detailed_forecasts = []
            for period in periods:
                number = period.get('number')
                detailed_forecast = period.get('detailedForecast', '')
                if number is not None:
                    detailed_forecasts.append({
                        "number": number,
                        "detailedForecast": detailed_forecast
                    })
            logging.debug(f"Extracted Detailed Forecasts: {detailed_forecasts}")
            return detailed_forecasts
        except KeyError as key_err:
            logging.error(f"Key error while extracting detailed forecasts: {key_err}")
            raise
        except Exception as e:
            logging.error(f"Error extracting detailed forecasts: {e}")
            raise

    def merge_detailed_forecasts(self, detailed_forecasts: List[Dict[str, str]]) -> str:
        """
        Merges the list of detailed forecasts into a structured string.

        Args:
            detailed_forecasts (list): List of dictionaries with 'number' and 'detailedForecast'.

        Returns:
            str: Merged structured string.
        """
        try:
            logging.info("Merging detailed forecasts into structured format.")
            merged_content = ""
            for forecast in detailed_forecasts:
                merged_content += f'"number": {forecast["number"]}\n"detailedForecast": "{forecast["detailedForecast"]}"\n'
            logging.debug(f"Merged Detailed Forecasts Content:\n{merged_content}")
            return merged_content
        except Exception as e:
            logging.error(f"Error merging detailed forecasts: {e}")
            raise

    ### New: Extract and Merge shortForecasts

    def extract_short_forecasts(self, forecast_data: dict) -> List[Dict[str, str]]:
        """
        Extracts the 'number' and 'shortForecast' from each forecast period.

        Args:
            forecast_data (dict): JSON data from NOAA NWS API.

        Returns:
            list: List of dictionaries containing 'number' and 'shortForecast'.
        """
        try:
            logging.info("Extracting short forecasts.")
            periods = forecast_data['properties']['periods']
            short_forecasts = []
            for period in periods:
                number = period.get('number')
                short_forecast = period.get('shortForecast', '')
                if number is not None:
                    short_forecasts.append({
                        "number": number,
                        "shortForecast": short_forecast
                    })
            logging.debug(f"Extracted Short Forecasts: {short_forecasts}")
            return short_forecasts
        except KeyError as key_err:
            logging.error(f"Key error while extracting short forecasts: {key_err}")
            raise
        except Exception as e:
            logging.error(f"Error extracting short forecasts: {e}")
            raise

    def merge_short_forecasts(self, short_forecasts: List[Dict[str, str]]) -> str:
        """
        Merges the list of short forecasts into a structured string.

        Args:
            short_forecasts (list): List of dictionaries with 'number' and 'shortForecast'.

        Returns:
            str: Merged structured string.
        """
        try:
            logging.info("Merging short forecasts into structured format.")
            merged_content = ""
            for forecast in short_forecasts:
                merged_content += f'"number": {forecast["number"]}\n"shortForecast": "{forecast["shortForecast"]}"\n'
            logging.debug(f"Merged Short Forecasts Content:\n{merged_content}")
            return merged_content
        except Exception as e:
            logging.error(f"Error merging short forecasts: {e}")
            raise

# ---------------------- Weather Summarizer ----------------------

class WeatherSummarizer:
    """
    Generates concise summaries from parsed weather forecast data using a summarization model.
    """

    def __init__(self, model_name: str = "facebook/bart-large-cnn", device: int = -1):
        """
        Initializes the summarizer with the specified model and device.

        Args:
            model_name (str): Name of the Hugging Face model to use.
            device (int): Device to run the model on (-1 for CPU, 0 for GPU).
        """
        try:
            logging.info("Loading spaCy language model for summarization.")
            self.nlp = spacy.load("en_core_web_trf")
            logging.info("spaCy model loaded successfully.")
        except Exception as e:
            logging.error(f"Failed to load spaCy model: {e}")
            raise

        try:
            logging.info(f"Loading BART summarization pipeline with model '{model_name}'.")
            self.summarizer = pipeline("summarization", model=model_name, device=device)
            logging.info("BART summarization pipeline loaded successfully.")
        except Exception as e:
            logging.error(f"Failed to load BART model: {e}")
            raise

    def summarize(self, paragraphs: List[str], max_length: int = 150, min_length: int = 40) -> str:
        """
        Generates a summary from the provided paragraphs.

        Args:
            paragraphs (list): List of descriptive paragraphs.
            max_length (int): Maximum length of the summary.
            min_length (int): Minimum length of the summary.

        Returns:
            str: Generated summary.
        """
        try:
            logging.info("Generating summary using BART model.")
            full_text = " ".join(paragraphs)
            summary = self.summarizer(
                full_text,
                max_length=max_length,
                min_length=min_length,
                do_sample=False
            )
            logging.info("Summary generated successfully.")
            return summary[0]['summary_text']
        except Exception as e:
            logging.error(f"Error during summarization: {e}")
            raise

    ### New: Summarize Merged Detailed Forecasts

    def summarize_merged_forecast(self, merged_content: str, max_length: int = 150, min_length: int = 40) -> str:
        """
        Generates a summary from the merged detailed forecasts.

        Args:
            merged_content (str): Merged detailed forecasts as a single string.
            max_length (int): Maximum length of the summary.
            min_length (int): Minimum length of the summary.

        Returns:
            str: Generated summary.
        """
        try:
            logging.info("Generating summary for merged detailed forecasts.")
            # Process the merged content with spaCy
            doc = self.nlp(merged_content)
            sentences = [sent.text for sent in doc.sents]
            paragraph = " ".join(sentences)
            logging.debug(f"Processed Paragraph for Summarization: {paragraph}")

            # Generate summary
            summary = self.summarizer(
                paragraph,
                max_length=max_length,
                min_length=min_length,
                do_sample=False
            )
            logging.info("Merged summary generated successfully.")
            return summary[0]['summary_text']
        except Exception as e:
            logging.error(f"Error during merged summarization: {e}")
            raise

    ### New: Summarize Merged Short Forecasts

    def summarize_merged_short_forecasts(self, merged_content: str, max_length: int = 150, min_length: int = 40) -> str:
        """
        Generates a summary from the merged short forecasts.

        Args:
            merged_content (str): Merged short forecasts as a single string.
            max_length (int): Maximum length of the summary.
            min_length (int): Minimum length of the summary.

        Returns:
            str: Generated summary.
        """
        try:
            logging.info("Generating summary for merged short forecasts.")
            # Process the merged content with spaCy
            doc = self.nlp(merged_content)
            sentences = [sent.text for sent in doc.sents]
            paragraph = " ".join(sentences)
            logging.debug(f"Processed Paragraph for Short Forecast Summarization: {paragraph}")

            # Generate summary
            summary = self.summarizer(
                paragraph,
                max_length=max_length,
                min_length=min_length,
                do_sample=False
            )
            logging.info("Short forecasts summary generated successfully.")
            return summary[0]['summary_text']
        except Exception as e:
            logging.error(f"Error during short forecasts summarization: {e}")
            raise

# ---------------------- Visualization ----------------------

def plot_temperatures(paragraphs: List[str]) -> Optional[plt.Figure]:
    """
    Generates a temperature trend plot from the forecast paragraphs.

    Args:
        paragraphs (list): List of descriptive paragraphs.

    Returns:
        matplotlib.figure.Figure: Generated plot figure, or None if no data.
    """
    temperatures = []
    periods = []
    for p in paragraphs:
        try:
            temp_start = p.find("temperature will be ") + len("temperature will be ")
            temp_end = p.find("°", temp_start)
            temp = int(p[temp_start:temp_end])
            name = p.split(":")[0]
            temperatures.append(temp)
            periods.append(name)
        except:
            continue

    if not temperatures:
        logging.warning("No temperature data available for plotting.")
        return None

    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(periods, temperatures, marker='o', linestyle='-', color='b')
    ax.set_title('Temperature Over Time')
    ax.set_xlabel('Period')
    ax.set_ylabel('Temperature (°F)')
    ax.grid(True)
    plt.tight_layout()
    return fig

# ---------------------- Input Validation ----------------------

def validate_coordinates(latitude: float, longitude: float) -> Optional[str]:
    """
    Validates the latitude and longitude values.

    Args:
        latitude (float): Latitude value.
        longitude (float): Longitude value.

    Returns:
        str: Error message if invalid, else None.
    """
    errors = []
    if not (-90 <= latitude <= 90):
        errors.append("Latitude must be between -90 and 90.")
    if not (-180 <= longitude <= 180):
        errors.append("Longitude must be between -180 and 180.")
    if errors:
        return " ".join(errors)
    return None

# ---------------------- Gradio Interface Function ----------------------

def weather_gradio_interface(latitude: float, longitude: float, show_plot: bool) -> Tuple[str, str, Optional[plt.Figure], str, str, str, str]:
    """
    Gradio interface function to fetch, parse, and summarize weather data.

    Args:
        latitude (float): Latitude input by the user.
        longitude (float): Longitude input by the user.
        show_plot (bool): Whether to display the temperature trend plot.

    Returns:
        tuple: Raw data (formatted JSON), summary, plot (if enabled), merged detailed forecasts, merged summary, merged short forecasts, merged short summary.
    """
    # Validate inputs
    validation_error = validate_coordinates(latitude, longitude)
    if validation_error:
        logging.error(f"Input validation failed: {validation_error}")
        return (
            validation_error,
            "Unable to generate summary due to invalid inputs.",
            None,
            "",
            "",
            "",
            ""
        )

    # Initialize components
    fetcher = NOAADataFetcher(user_agent=USER_AGENT)
    parser = WeatherDataParser()
    summarizer = WeatherSummarizer(model_name="facebook/bart-large-cnn", device=0 if torch.cuda.is_available() else -1)

    try:
        # Fetch data
        forecast_data = fetcher.fetch_forecast(latitude, longitude)
        if not forecast_data:
            logging.error("No forecast data retrieved.")
            return (
                "No forecast data retrieved.",
                "Unable to generate summary.",
                None,
                "",
                "",
                "",
                ""
            )

        # Parse data
        paragraphs = parser.parse_forecast(forecast_data, days=FORECAST_DAYS)
        if not paragraphs:
            logging.error("No weather information available to summarize.")
            return (
                "No weather information available to summarize.",
                "Unable to generate summary.",
                None,
                "",
                "",
                "",
                ""
            )

        # Summarize
        summary = summarizer.summarize(paragraphs)

        # Prepare raw data
        raw_data = json.dumps(forecast_data, indent=2)

        # Plot temperatures if requested
        plot = plot_temperatures(paragraphs) if show_plot else None

        ### Existing: Extract and Merge Detailed Forecasts
        detailed_forecasts = parser.extract_detailed_forecasts(forecast_data)
        merged_detailed_forecasts = parser.merge_detailed_forecasts(detailed_forecasts)

        # Summarize merged detailed forecasts
        merged_summary = summarizer.summarize_merged_forecast(merged_detailed_forecasts)

        ### New: Extract and Merge Short Forecasts
        short_forecasts = parser.extract_short_forecasts(forecast_data)
        merged_short_forecasts = parser.merge_short_forecasts(short_forecasts)

        # Summarize merged short forecasts
        merged_short_summary = summarizer.summarize_merged_short_forecasts(merged_short_forecasts)

        return (
            raw_data,
            summary,
            plot,
            merged_detailed_forecasts,
            merged_summary,
            merged_short_forecasts,
            merged_short_summary
        )

    except Exception as e:
        logging.critical(f"Error in Gradio interface: {e}")
        return (
            f"An error occurred: {e}",
            "Unable to generate summary.",
            None,
            "",
            "",
            "",
            ""
        )

# ---------------------- Main Execution ----------------------

if __name__ == "__main__":
    # Ensure Hugging Face Token is set if available
    if HF_TOKEN:
        os.environ["HUGGINGFACE_HUB_TOKEN"] = HF_TOKEN
        logging.info("Hugging Face token found and set.")
    else:
        logging.warning("""
        Hugging Face token (HF_TOKEN) not found.
        Proceeding without authentication may lead to rate limiting.
        To set the token, add it to environment variables or set it directly in the notebook.
        """)

    # Determine device for transformers pipeline
    import torch
    device = 0 if torch.cuda.is_available() else -1
    if device == 0:
        logging.info("Using GPU for transformers pipeline.")
    else:
        logging.info("GPU not available. Using CPU for transformers pipeline.")

    # Launch Gradio Interface
    with gr.Blocks() as demo:
        gr.Markdown("# Weather Summarizer")
        gr.Markdown("""
        Enter the **latitude** and **longitude** of a location to retrieve a summarized weather forecast.
        """)

        with gr.Row():
            latitude_input = gr.Number(label="Latitude", value=32.7767)
            longitude_input = gr.Number(label="Longitude", value=-96.7970)

        show_plot = gr.Checkbox(label="Show Temperature Trend Plot", value=True)

        submit_button = gr.Button("Submit")

        with gr.Tab("Raw Data"):
            raw_output = gr.Textbox(label="Raw NOAA API Response", lines=20, interactive=False)

        with gr.Tab("Raw Summary"):
            summary_output = gr.Textbox(label="Summarized Weather Forecast", lines=10, interactive=False)

        with gr.Tab("Temperature Plot"):
            temperature_plot = gr.Plot(label="Temperature Trend")

        with gr.Tab("Detailed Forecasts"):
            merged_forecast_output = gr.Textbox(label="Merged Detailed Forecasts", lines=30, interactive=False)

        with gr.Tab("Detailed Summary"):
            merged_summary_output = gr.Textbox(label="Summarized Merged Forecast", lines=5, interactive=False)

        ### New: Additional Tabs for Short Forecasts and Summary
        with gr.Tab("Short Forecasts"):
            merged_short_forecast_output = gr.Textbox(label="Merged Short Forecasts", lines=30, interactive=False)

        with gr.Tab("Short Summary"):
            merged_short_summary_output = gr.Textbox(label="Summarized Short Forecasts", lines=5, interactive=False)

        submit_button.click(
            fn=weather_gradio_interface,
            inputs=[latitude_input, longitude_input, show_plot],
            outputs=[
                raw_output,
                summary_output,
                temperature_plot,
                merged_forecast_output,
                merged_summary_output,
                merged_short_forecast_output,
                merged_short_summary_output
            ]
        )

        gr.Markdown("""
        ---
        **Note:** Ensure that the latitude and longitude values are valid. For example:
        - **Dallas, TX:** Latitude `32.7767`, Longitude `-96.7970`
        - **New York City, NY:** Latitude `40.7128`, Longitude `-74.0060`
        """)

    demo.launch()


        Hugging Face token (HF_TOKEN) not found.
        Proceeding without authentication may lead to rate limiting.
        To set the token, add it to environment variables or set it directly in the notebook.
        


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b99718d1c2cbded0ac.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [65]:
!pip install openai

In [72]:
#### NOAA API WEATHER FORECAST SUMMARIZATION
# NOAA Raw Response + Sumarize weather forecast
# Detailed forecast + detailed summary
# Short forecat + short summary
######## OpenaAI summarization

import os
import json
import logging
from typing import Optional, Tuple, List, Dict
from datetime import datetime

import requests
from cachetools import TTLCache, cached
import spacy
import gradio as gr

# NEW (OpenAI v1 migration)
from openai import OpenAI
# Hardcode your API key here
client = OpenAI(
    api_key="YOUR_OPENAI_API_KEY"  # <-- Replace with your actual key
)

# ---------------------- Configuration ----------------------

USER_AGENT = "WeatherApp/1.0 (your.email@example.com)"  # Replace with your actual email
CACHE_TTL = 600  # Cache Time-To-Live in seconds (e.g., 10 minutes)
FORECAST_DAYS = 14  # Number of days to include in the summary

# Optional: Hugging Face Token for authenticated access
HF_TOKEN = os.getenv("HF_TOKEN")  # Ensure this environment variable is set if you have a token

# ---------------------- Logging Setup ----------------------

def setup_logging():
    """
    Configures the logging settings.
    """
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s'
    )

setup_logging()

# ---------------------- Cache Setup ----------------------

# Initialize a TTL cache
forecast_cache = TTLCache(maxsize=100, ttl=CACHE_TTL)

# ---------------------- NOAA Data Fetcher ----------------------

class NOAADataFetcher:
    """
    Fetches weather forecast data from NOAA's National Weather Service (NWS) API.
    """

    def __init__(self, user_agent: str):
        self.user_agent = user_agent
        self.headers = {'User-Agent': self.user_agent}
        self.endpoint_template = "https://api.weather.gov/points/{latitude},{longitude}"

    @cached(cache=forecast_cache)
    def fetch_forecast(self, latitude: float, longitude: float) -> Optional[dict]:
        """
        Fetches the weather forecast data for the given latitude and longitude.
        """
        endpoint = self.endpoint_template.format(latitude=latitude, longitude=longitude)
        try:
            logging.info(f"Fetching metadata from NOAA NWS API for lat: {latitude}, lon: {longitude}.")
            response = requests.get(endpoint, headers=self.headers, timeout=10)
            response.raise_for_status()
            data = response.json()
            logging.debug(f"Metadata Response: {json.dumps(data, indent=2)}")

            # Extract the forecast URL
            forecast_url = data['properties']['forecast']
            logging.info("Fetching forecast data from NOAA NWS API.")
            forecast_response = requests.get(forecast_url, headers=self.headers, timeout=10)
            forecast_response.raise_for_status()
            forecast_data = forecast_response.json()
            logging.debug(f"Forecast Response: {json.dumps(forecast_data, indent=2)}")

            return forecast_data

        except requests.exceptions.HTTPError as http_err:
            logging.error(f"HTTP error occurred: {http_err}")
        except requests.exceptions.RequestException as req_err:
            logging.error(f"Request exception: {req_err}")
        except KeyError as key_err:
            logging.error(f"Key error: {key_err}")
        except Exception as e:
            logging.error(f"An unexpected error occurred: {e}")

        return None

# ---------------------- Weather Data Parser ----------------------

class WeatherDataParser:
    """
    Parses and processes raw weather forecast data from NOAA's NWS API.
    """

    def parse_forecast(self, forecast_data: dict, days: int = 14) -> List[str]:
        """
        Parses forecast data and constructs descriptive paragraphs.
        """
        try:
            logging.info("Parsing weather data.")
            periods = forecast_data['properties']['periods']
            filtered_periods = self.filter_forecast_periods(periods, days)
            paragraphs = self.construct_paragraphs(filtered_periods)
            logging.debug(f"Full Paragraph: {' '.join(paragraphs)}")
            return paragraphs
        except KeyError as key_err:
            logging.error(f"Key error while parsing weather data: {key_err}")
            raise
        except Exception as e:
            logging.error(f"Error parsing weather data: {e}")
            raise

    def filter_forecast_periods(self, periods: List[dict], days: int) -> List[dict]:
        """
        Filters forecast periods to include only the specified number of days.
        """
        filtered = []
        current_day = None
        for period in periods:
            day = period['startTime'][:10]  # Extract YYYY-MM-DD
            if day != current_day:
                current_day = day
                filtered.append(period)
                if len(filtered) >= days * 2:  # Day and Night for each day
                    break
        logging.info(f"Filtered down to {len(filtered)} periods for summarization.")
        return filtered

    def construct_paragraphs(self, periods: List[dict]) -> List[str]:
        """
        Constructs descriptive paragraphs from forecast periods.
        """
        paragraphs = []
        for period in periods:
            name = period.get('name', 'N/A')
            temperature = period.get('temperature', 'N/A')
            temperature_unit = period.get('temperatureUnit', '')
            wind_speed = period.get('windSpeed', 'N/A')
            wind_direction = period.get('windDirection', 'N/A')
            short_forecast = period.get('shortForecast', '')
            detailed_forecast = period.get('detailedForecast', '')

            forecast_paragraph = (
                f"{name}: The temperature will be {temperature}°{temperature_unit} "
                f"with {short_forecast.lower()}. Winds are expected to be {wind_speed} "
                f"from the {wind_direction.lower()}. {detailed_forecast}"
            )
            paragraphs.append(forecast_paragraph)
            logging.debug(f"Parsed forecast for {name}: {forecast_paragraph}")
        return paragraphs

    def extract_and_merge_detailed_forecasts(self, forecast_data: dict) -> str:
        """
        Extracts 'number' and 'detailedForecast', transforms them, and concatenates into a paragraph.
        """
        try:
            logging.info("Extracting and merging detailed forecasts.")
            periods = forecast_data['properties']['periods']
            merged_content = ""
            for period in periods:
                number = period.get('number')
                detailed_forecast = period.get('detailedForecast', '').strip()
                if number is not None and detailed_forecast:
                    day_forecast = f"day {number} forecast suggests {detailed_forecast}"
                    merged_content += f"{day_forecast} "
            merged_content = merged_content.strip()
            logging.debug(f"Merged Detailed Forecasts Paragraph: {merged_content}")
            return merged_content
        except KeyError as key_err:
            logging.error(f"Key error while extracting detailed forecasts: {key_err}")
            raise
        except Exception as e:
            logging.error(f"Error extracting detailed forecasts: {e}")
            raise

    def extract_and_merge_short_forecasts(self, forecast_data: dict) -> str:
        """
        Extracts 'number' and 'shortForecast', transforms them, and concatenates into a paragraph.
        """
        try:
            logging.info("Extracting and merging short forecasts.")
            periods = forecast_data['properties']['periods']
            merged_content = ""
            for period in periods:
                number = period.get('number')
                short_forecast = period.get('shortForecast', '').strip()
                if number is not None and short_forecast:
                    day_forecast = f"day {number} forecast suggests {short_forecast}"
                    merged_content += f"{day_forecast} "
            merged_content = merged_content.strip()
            logging.debug(f"Merged Short Forecasts Paragraph: {merged_content}")
            return merged_content
        except KeyError as key_err:
            logging.error(f"Key error while extracting short forecasts: {key_err}")
            raise
        except Exception as e:
            logging.error(f"Error extracting short forecasts: {e}")
            raise

# ---------------------- Weather Summarizer ----------------------

class WeatherSummarizer:
    """
    Generates summaries using OpenAI GPT-4o-mini (via the new v1 migration).
    """

    def __init__(self):
        """
        Initializes the summarizer.
        Loads spaCy for potential text processing if needed.
        """
        try:
            logging.info("Loading spaCy language model.")
            self.nlp = spacy.load("en_core_web_trf")
            logging.info("spaCy model loaded successfully.")
        except Exception as e:
            logging.error(f"Failed to load spaCy model: {e}")
            raise

    def summarize(self, paragraphs: List[str], max_length: int = 150, min_length: int = 40) -> str:
        """
        (Optional) Summarize a list of paragraphs using GPT-4o-mini.
        """
        try:
            logging.info("Generating summary with GPT-4o-mini (optional).")
            full_text = " ".join(paragraphs)

            # Use the new openai v1 style calls
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {
                        "role": "system",
                        "content": "You are a helpful assistant summarizing weather data."
                    },
                    {
                        "role": "user",
                        "content": full_text
                    }
                ],
                temperature=0,
            )
            return response.choices[0].message.content
        except Exception as e:
            logging.error(f"Error during summarization: {e}")
            raise

    def summarize_merged_detailed_forecasts(self, merged_content: str, max_length: int = 700, min_length: int = 40) -> str:
        """
        Generates a summary from the merged detailed forecasts using GPT-4o-mini.
        """
        try:
            logging.info("Generating GPT-4o-mini summary for merged detailed forecasts.")

            doc = self.nlp(merged_content)
            sentences = [sent.text for sent in doc.sents]
            paragraph = " ".join(sentences)

            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {
                        "role": "system",
                        "content": "You are a helpful assistant summarizing weather forecasts in detail."
                    },
                    {
                        "role": "user",
                        "content": paragraph
                    }
                ],
                temperature=0,
            )
            return response.choices[0].message.content
        except Exception as e:
            logging.error(f"Error during detailed forecasts summarization: {e}")
            raise

    def summarize_merged_short_forecasts(self, merged_content: str, max_length: int = 150, min_length: int = 40) -> str:
        """
        Generates a summary from the merged short forecasts using GPT-4o-mini.
        """
        try:
            logging.info("Generating GPT-4o-mini summary for merged short forecasts.")

            doc = self.nlp(merged_content)
            sentences = [sent.text for sent in doc.sents]
            paragraph = " ".join(sentences)

            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {
                        "role": "system",
                        "content": "You are a helpful assistant summarizing short weather forecasts."
                    },
                    {
                        "role": "user",
                        "content": paragraph
                    }
                ],
                temperature=0,
            )
            return response.choices[0].message.content
        except Exception as e:
            logging.error(f"Error during short forecasts summarization: {e}")
            raise

# ---------------------- Input Validation ----------------------

def validate_coordinates(latitude: float, longitude: float) -> Optional[str]:
    """
    Validates the latitude and longitude values.
    """
    errors = []
    if not (-90 <= latitude <= 90):
        errors.append("Latitude must be between -90 and 90.")
    if not (-180 <= longitude <= 180):
        errors.append("Longitude must be between -180 and 180.")
    if errors:
        return " ".join(errors)
    return None

# ---------------------- Gradio Interface Function ----------------------

def weather_gradio_interface(latitude: float, longitude: float) -> Tuple[str, str, str, str, str, str]:
    """
    Gradio interface function to fetch, parse, and summarize weather data.
    Returns:
        tuple:
            - Raw NOAA API response (as text)
            - Summary of forecast
            - Merged Detailed Forecasts
            - Summarized Detailed Forecast
            - Merged Short Forecasts
            - Summarized Short Forecast
    """
    # Validate inputs
    validation_error = validate_coordinates(latitude, longitude)
    if validation_error:
        logging.error(f"Input validation failed: {validation_error}")
        return (
            validation_error,
            "Unable to generate summary due to invalid inputs.",
            "",
            "",
            "",
            ""
        )

    # Initialize components
    fetcher = NOAADataFetcher(user_agent=USER_AGENT)
    parser = WeatherDataParser()
    summarizer = WeatherSummarizer()

    try:
        # Fetch data
        forecast_data = fetcher.fetch_forecast(latitude, longitude)
        if not forecast_data:
            logging.error("No forecast data retrieved.")
            return (
                "No forecast data retrieved.",
                "Unable to generate summary.",
                "",
                "",
                "",
                ""
            )

        # Parse data
        paragraphs = parser.parse_forecast(forecast_data, days=FORECAST_DAYS)
        if not paragraphs:
            logging.error("No weather information available to summarize.")
            return (
                "No weather information available to summarize.",
                "Unable to generate summary.",
                "",
                "",
                "",
                ""
            )

        # Summarize (using GPT-4o-mini)
        summary = summarizer.summarize(paragraphs)

        # Prepare raw data
        raw_data = json.dumps(forecast_data, indent=2)

        # Extract/Merge & Summarize Detailed Forecasts
        merged_detailed_forecasts = parser.extract_and_merge_detailed_forecasts(forecast_data)
        merged_detailed_summary = summarizer.summarize_merged_detailed_forecasts(merged_detailed_forecasts)

        # Extract/Merge & Summarize Short Forecasts
        merged_short_forecasts = parser.extract_and_merge_short_forecasts(forecast_data)
        merged_short_summary = summarizer.summarize_merged_short_forecasts(merged_short_forecasts)

        return (
            raw_data,
            summary,
            merged_detailed_forecasts,
            merged_detailed_summary,
            merged_short_forecasts,
            merged_short_summary
        )

    except Exception as e:
        logging.critical(f"Error in Gradio interface: {e}")
        return (
            f"An error occurred: {e}",
            "Unable to generate summary.",
            "",
            "",
            "",
            ""
        )

# ---------------------- Main Execution ----------------------

if __name__ == "__main__":
    if HF_TOKEN:
        os.environ["HUGGINGFACE_HUB_TOKEN"] = HF_TOKEN
        logging.info("Hugging Face token found and set.")
    else:
        logging.warning("""
        Hugging Face token (HF_TOKEN) not found.
        Proceeding without authentication may lead to rate limiting.
        """)

    import torch
    device = 0 if torch.cuda.is_available() else -1
    if device == 0:
        logging.info("GPU is available, though not used by GPT-4o-mini here.")
    else:
        logging.info("Using CPU for summarization calls to GPT-4o-mini.")

    with gr.Blocks() as demo:
        gr.Markdown("# NOAA API Weather Forecast Summarizer by usig Spacy and OpenAI")
        gr.Markdown("""
        Developed by **Partha Pratim Ray**, **parthapratimray1986@gmail.com**
        """)
        gr.Markdown("""
        Enter the **latitude** and **longitude** of a location to retrieve a summarized weather forecast.
        """)

        with gr.Row():
            latitude_input = gr.Number(label="Latitude", value=32.7767)
            longitude_input = gr.Number(label="Longitude", value=-96.7970)

        submit_button = gr.Button("Submit")

        with gr.Tab("Raw Data"):
            raw_output = gr.Textbox(label="Raw NOAA API Response", lines=20, interactive=False)

        with gr.Tab("Summary"):
            summary_output = gr.Textbox(label="Summarized Weather Forecast", lines=10, interactive=False)

        with gr.Tab("Detailed Forecasts"):
            merged_detailed_forecast_output = gr.Textbox(label="Merged Detailed Forecasts", lines=30, interactive=False)

        with gr.Tab("Detailed Forecast Summary"):
            merged_detailed_summary_output = gr.Textbox(label="Summarized Detailed Forecast", lines=5, interactive=False)

        with gr.Tab("Short Forecasts"):
            merged_short_forecast_output = gr.Textbox(label="Merged Short Forecasts", lines=30, interactive=False)

        with gr.Tab("Short Forecast Summary"):
            merged_short_summary_output = gr.Textbox(label="Summarized Short Forecasts", lines=5, interactive=False)

        submit_button.click(
            fn=weather_gradio_interface,
            inputs=[latitude_input, longitude_input],
            outputs=[
                raw_output,
                summary_output,
                merged_detailed_forecast_output,
                merged_detailed_summary_output,
                merged_short_forecast_output,
                merged_short_summary_output
            ]
        )

        gr.Markdown("""
        ---
        **Note:** Ensure that the latitude and longitude values are valid. For example:
        - **Dallas, TX:** Latitude `32.7767`, Longitude `-96.7970`
        - **New York City, NY:** Latitude `40.7128`, Longitude `-74.0060`
        """)

    demo.launch()


        Hugging Face token (HF_TOKEN) not found.
        Proceeding without authentication may lead to rate limiting.
        


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2371b00d78a3d52ecb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
